In [1]:
from Parameterize_and_testing_functions import *

## =====================
There does not seem to be any diiference across box sizes, 
so we go with 15x15 nm and to start 3-5 reps. 

Use DSPC:CHOL in 100:0, 90:10, 80:20 

You can then use DSPC:ST same ratio with the other sterols, or you can try with DPPC as with and then only with 5 % ST, and see that the TM will shift depending on the sterol added.

NB I had to set -DFLEXIBLE during relaxation for CAMP. This was not done for the other sterols

## ======================

In [2]:
def prepare_setup_hexa(ST, mdp_loc='/data2/lisbeth_is_awesome/STEROLS/TMs/files', lipids='POPC DPPC DIPC DPSM CHOL DBPC DOPC'):
    '''Prepare setup of a hexagonal box for testing TMs of the sterols. Set ST to CHOL for cholesterol, or else for other sterols'''
    ###fix topology
    itpDir='/data1/lisbeth/Martini_ITPs'
    ffbonded = '/data1/lisbeth/Martini_ITPs/martini3-lipids-master/top/martini_v3.0_ffbonded_dev_v18.itp'
    CHOL='/data1/lisbeth/Martini_ITPs/DEV19_CHOL/martini_v3.0_sterols_dev20.itp'
    #STs = '/data1/lisbeth/ILs_M3/STEROLS/martini_v3.0_fancysterols_dev0-0-1.itp'
    STs = '/data1/lisbeth/PYTHON/Martini_FF_initative_ILs/M3-Ionizable-Lipids/Collection_of_itps/Sterols/martini_v3.0_fancysterols_dev0-0-4.itp'

    if ST == 'CHOL':
        with open('topol.top') as topinput:
            top = topinput.readlines()
        top[0]= f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{itpDir}/martini_v3.0.0_phospholipids_v1.itp"\n\
        #include "{CHOL}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n'
        with open('topol.top', 'w+') as topout:
            for line in top:
                topout.write(line)
    else:
        with open('topol.top') as topinput:
            top = topinput.readlines()
        top[0]= f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{ffbonded}"\n#include "{itpDir}/martini_v3.0.0_phospholipids_v1.itp"\n#include \
        "{STs}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n'
        with open('topol.top', 'w+') as topout:
            for line in top:
                topout.write(line)

    ### Neutralize
    #print('gmx grompp -f {}/min.mdp -c mem.gro -p topol.top -o memion.tpr -maxwarn 1'.format(mdp_loc))
    subprocess.call('gmx grompp -f {}/minim.mdp -c mem.gro -p topol.top -o memion.tpr -r mem.gro -maxwarn 3'.format(mdp_loc)
                , shell=True)
    #print ('gmx genion -s memion.tpr -o memion.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -neutral')
    p = subprocess.Popen('gmx genion -s memion.tpr -o memion.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -neutral'
                         , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### add 150mM NaCl
    Waterline =  top[-1].split(' ')
    Waternumber = int( Waterline[-1].rstrip() )
    naclNUM = int((0.15 * Waternumber*4)/55.5)
    subprocess.call('gmx grompp -f {}/minim.mdp -c memion.gro -r memion.gro -p topol.top -o memion_2.tpr -maxwarn 3'.format(mdp_loc)
                , shell=True)
    p = subprocess.Popen('gmx genion -s memion_2.tpr -o memion_2.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -np {} -nn {}'.format(naclNUM, naclNUM)
                         , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### Minimize the system
    subprocess.call("gmx grompp -f {}/minim.mdp -c memion_2.gro -r memion_2.gro -p topol.top -o m.tpr -maxwarn 3".format(mdp_loc)
                    , shell = True)
    subprocess.call("gmx mdrun -v -deffnm m -nt 5 -pin on -pinstride 1 -pinoffset 0"
                    , shell = True)

    ### Create the index file
    gro = 'm.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('not resname {}'.format(lipids))
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'Solvent')
    u.atoms.write("index.ndx", mode="a", name= 'System')

    ### Relax the system
    subprocess.call("gmx grompp -f {}/equil.mdp -c m.gro -r m.gro -p topol.top -o r.tpr -n index.ndx -maxwarn 1".format(mdp_loc)
                     , shell = True)
    subprocess.call("gmx mdrun -v -deffnm r -nt 10 -pin on -pinstride 1 -pinoffset 0"
                  , shell = True)

    ### Prepare the simulation
    subprocess.call("gmx grompp -f {}/prod_anneal.mdp -c r.gro -p topol.top -r m.gro -o p.tpr -n index.ndx -maxwarn 1".format( mdp_loc)
                      , shell = True)
    return


In [3]:
def prepare_setup_hexa_DEV17(ST, mdp_loc='/data2/lisbeth_is_awesome/STEROLS/TMs/files', lipids='POPC DPPC DIPC DPSM CHOL DBPC DOPC'):
    '''Prepare setup of a hexagonal box for testing TMs of the sterols. Set ST to CHOL for cholesterol, or else for other sterols'''
    ###fix topology
    itpDir='/data1/lisbeth/Martini_ITPs/martini3-lipids-master/top'
    CHOL='/data1/lisbeth/Martini_ITPs/DEV19_CHOL/martini_v3.0_sterols_dev20.itp'
    STs = '/data1/lisbeth/Martini_ITPs/fancy_sterols/martini_v3.0_fancysterols_dev0-0-3.itp'

    if ST == 'CHOL':
        with open('topol.top') as topinput:
            top = topinput.readlines()
        top[0]= f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{itpDir}/martini_v3.0_ffbonded_dev_v17.itp"\n#include "{itpDir}/martini_v3.0_phospholipids_ALL_dev_v17.itp"\n#include "{CHOL}"\n\
        #include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n'
        with open('topol.top', 'w+') as topout:
            for line in top:
                topout.write(line)
    else:
        with open('topol.top') as topinput:
            top = topinput.readlines()
        top[0]= f'#include "{itpDir}/martini_v3.0.0.itp"\n#include "{itpDir}/martini_v3.0_ffbonded_dev_v17.itp"\n#include "{itpDir}/martini_v3.0_phospholipids_ALL_dev_v17.itp"\n#include \
        "{STs}"\n#include "{itpDir}/martini_v3.0.0_solvents_v1.itp"\n#include "{itpDir}/martini_v3.0.0_ions_v1.itp"\n'
        with open('topol.top', 'w+') as topout:
            for line in top:
                topout.write(line)

    ### Neutralize
    #print('gmx grompp -f {}/min.mdp -c mem.gro -p topol.top -o memion.tpr -maxwarn 1'.format(mdp_loc))
    subprocess.call('gmx grompp -f {}/minim.mdp -c mem.gro -p topol.top -o memion.tpr -r mem.gro -maxwarn 3'.format(mdp_loc)
                , shell=True)
    #print ('gmx genion -s memion.tpr -o memion.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -neutral')
    p = subprocess.Popen('gmx genion -s memion.tpr -o memion.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -neutral'
                         , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### add 150mM NaCl
    Waterline =  top[-1].split(' ')
    Waternumber = int( Waterline[-1].rstrip() )
    naclNUM = int((0.15 * Waternumber*4)/55.5)
    subprocess.call('gmx grompp -f {}/minim.mdp -c memion.gro -r memion.gro -p topol.top -o memion_2.tpr -maxwarn 3'.format(mdp_loc)
                , shell=True)
    p = subprocess.Popen('gmx genion -s memion_2.tpr -o memion_2.gro -p topol.top -pname NA -pq +1 -nname CL -nq -1 -np {} -nn {}'.format(naclNUM, naclNUM)
                         , stdin=subprocess.PIPE, shell=True, universal_newlines=True)
    p.communicate('W')
    p.wait()

    ### Minimize the system
    subprocess.call("gmx grompp -f {}/minim.mdp -c memion_2.gro -r memion_2.gro -p topol.top -o m.tpr -maxwarn 3".format(mdp_loc)
                    , shell = True)
    subprocess.call("gmx mdrun -v -deffnm m -nt 5"
                    , shell = True)

    ### Create the index file
    gro = 'm.gro'
    u = md.Universe(gro)
    lipidsagg=u.select_atoms('resname {}'.format(lipids))
    solventagg=u.select_atoms('not resname {}'.format(lipids))
    lipidsagg.write("index.ndx", mode="w", name= 'Lipids')
    solventagg.write("index.ndx", mode="a", name= 'Solvent')
    u.atoms.write("index.ndx", mode="a", name= 'System')

    ### Relax the system
    subprocess.call("gmx grompp -f {}/equil.mdp -c m.gro -r m.gro -p topol.top -o r.tpr -n index.ndx -maxwarn 1".format(mdp_loc)
                     , shell = True)
    subprocess.call("gmx mdrun -v -deffnm r -nt 10 -pin on -pinstride 1 -pinoffset 0"
                  , shell = True)

    ### Prepare the simulation
    subprocess.call("gmx grompp -f {}/prod_anneal.mdp -c r.gro -p topol.top -r m.gro -o p.tpr -n index.ndx -maxwarn 1".format( mdp_loc)
                      , shell = True)
    return


In [4]:
main_dir = '/data2/lisbeth_is_awesome/STEROLS'
mdp_loc_BL = '/data1/lisbeth/Params/BILAYERS/MDPs'
mdp_loc_temp = '/data2/lisbeth_is_awesome/STEROLS/TMs/files'
ILs_itps = '/data1/lisbeth/ILs_M3'
CHOL_19_itp = '/data1/lisbeth/Martini_ITPs/DEV19_CHOL'
fancy_sterols = '/data1/lisbeth/ILs_M3/STEROLS'
out_put_dir_main = '/data2/lisbeth_is_awesome/STEROLS/TMs/VER20_CAMP'
insane_altered = '/data1/lisbeth/ILs_M3/STEROLS'
insane = '/data1/lisbeth/PYTHON'
Sterols = ['CHOL', 'ERG', 'SITO', 'STIG', 'CAMP']

## DSPC 10x10 Hexagonal box

In [ ]:
PC_comp = 100
ST_comp = 0

PC = 'DSPC'
ST = 'CHOL'

os.chdir(out_put_dir_main)
outdir = '{}_{}_{}_{}_box_10_10'.format(PC, PC_comp,ST, ST_comp)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)


print ('Folder:', outdir)

print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


prepare_setup_hexa(ST, lipids='DSPC')

## DSPC 15x15 Hexagonal box

In [ ]:
PC_comp = 100
ST_comp = 0

PC = 'DSPC'
ST = 'CHOL'

rep = 5

for r in range(1,rep):
    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)


    print ('Folder:', outdir)

    print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
    os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


    prepare_setup_hexa(ST, lipids='DSPC')

## DSPC 20x20 hexagonal box

In [ ]:
PC_comp = 100
ST_comp = 0

PC = 'DSPC'
ST = 'CHOL'

os.chdir(out_put_dir)
outdir = '{}_{}_{}_{}_box_20_20'.format(PC, PC_comp,ST, ST_comp)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)


print ('Folder:', outdir)

print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


prepare_setup_hexa(ST, lipids='DSPC')

## Cholesterol 10% 10x10 hexagonal box

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'CHOL'

rep = 5

for i in range(1,rep):
    os.chdir(out_put_dir)
    outdir = '{}_{}_{}_{}_box_10_10_rep{}'.format(PC, PC_comp,ST, ST_comp, rep)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)


    print ('Folder:', outdir)

    print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
    os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


    prepare_setup_hexa(ST)

## Cholesterol 20% 10x10 hexagonal box

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'CHOL'

os.chdir(out_put_dir)
outdir = '{}_{}_{}_{}_box_10_10'.format(PC, PC_comp,ST, ST_comp)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)


print ('Folder:', outdir)

print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 10 -y 10 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


prepare_setup_hexa(ST)

## Cholesterol 10% 15x15 hexagonal box

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'CHOL'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
    os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


    prepare_setup_hexa(ST)

## Cholesterol 20% 15x15 hexagonal box

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'CHOL'

rep = 5 

for r in range(1,rep):
    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp, r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)


    print ('Folder:', outdir)

    print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
    os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


    prepare_setup_hexa(ST)

## Cholesterol 10% 20x20 hexagonal box

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'CHOL'

os.chdir(out_put_dir)
outdir = '{}_{}_{}_{}_box_20_20'.format(PC, PC_comp,ST, ST_comp)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)


print ('Folder:', outdir)

print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


prepare_setup_hexa(ST)

## Cholesterol 20% 20x20 hexagonal box

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'CHOL'

os.chdir(out_put_dir)
outdir = '{}_{}_{}_{}_box_20_20'.format(PC, PC_comp,ST, ST_comp)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)


print ('Folder:', outdir)

print ('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))
os.system('python2 {4}/insane_altered.py -l {0}:{1} -l {2}:{3} \
 -pbc hexagonal -x 20 -y 20 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane_altered))


prepare_setup_hexa(ST)

# Fancy sterols V1

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'ERG'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'ERG'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'SITO'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'SITO'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [ ]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'STIG'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [ ]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'STIG'

rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

In [7]:
PC_comp = 90
ST_comp = 10

PC = 'DSPC'
ST = 'CAMP'


rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

Folder: DSPC_90_CAMP_10_box_15_15_rep1
python2 /data1/lisbeth/PYTHON/insane.py -l DSPC:90 -l CAMP:10      -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top


; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 361 lipids in upper leaflet, 361 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 8496
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 8497 24185
; NDX System 1 24185
; "I mean, the good stuff is just INSANE" --Julia Ormond
                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul

Analysing residue names:
There are: 16409      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


No ions to add, will just copy input configuration.

Back Off! I just backed up memion.gro to ./#memion.gro.2#

GROMACS reminds you: "The use of COBOL cripples the mind; its teaching should therefore be regarded as a criminal offense." (Edsger Dijkstra)

                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pro

Analysing residue names:
There are: 16409      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


...

No protocol specified
Reading file m.tpr, VERSION 2020.7 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 5 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Step -1, time -0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.134676, max 0.472503 (between atoms 4011 and 4010)
bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length

Back Off! I just backed up m.trr to ./#m.trr.1#

Back Off! I just backed up m.edr to ./#m.edr.1#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =          500
Step=    0, Dmax= 1.0e-02 nm, Epot=  6.71108e+09 Fmax= 4.44825e+11, atom= 3918

Step 1, time 0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.0

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.323 nm, buffer size 0.223 nm
Set rlist, assuming 4x4 atom pair-list, to 1.228 nm, buffer size 0.128 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
This run will generate roughly 8 Mb of data


No protocol specified
Reading file r.tpr, VERSION 2020.7 (single precision)
Changing nstlist from 20 to 25, rlist from 1.228 to 1.282

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Back Off! I just backed up r.xtc to ./#r.xtc.1#

Back Off! I just backed up r.edr to ./#r.edr.1#
starting mdrun 'INSANE! Membrane UpperLeaflet>DSPC:CAMP=90.0:10.0 LowerLeaflet>DSPC:CAMP=90.0:10.0'
2500000 steps,  50000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.985215 0.985215 0.986589
step 2499900, remaining wall clock time:     0 s          
Writing final coordinates.
step 2500000, remaining wall clock time:     0 s          
NOTE: 12 % of the run time was spent in pair search,
      you might want to increase nstli

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.324 nm, buffer size 0.224 nm
Set rlist, assuming 4x4 atom pair-list, to 1.228 nm, buffer size 0.128 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup


Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to 1811939070
Velocities were taken from a Maxwell distribution at 373 K
Cleaning up constraints and constant bonded interactions with virtual sites
Simulated annealing for group Lipids: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Simulated annealing for group Solvent: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Number of degrees of freedom in T-Coupling group Lipids is 24189.00
Number of degrees of freedom in T-Coupling group Solv

In [8]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'CAMP'


rep = 5

for r in range(1,rep):

    os.chdir(out_put_dir_main)
    outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
    if os.path.isdir(outdir)==False:
        os.mkdir(outdir)
        os.chdir(outdir)
    elif os.path.isdir(outdir)==True:
        os.chdir(outdir)

    print ('Folder:', outdir)

    print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
    os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


    prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

Folder: DSPC_80_CAMP_20_box_15_15_rep1
python2 /data1/lisbeth/PYTHON/insane.py -l DSPC:80 -l CAMP:20      -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top


; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 361 lipids in upper leaflet, 361 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 8352
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 8353 23975
; NDX System 1 23975
; "I mean, the good stuff is just INSANE" --Julia Ormond
                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul

Analysing residue names:
There are: 16343      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


No ions to add, will just copy input configuration.

GROMACS reminds you: "Science adjusts its views based on what's observed. Faith is the denial of observation so that belief can be preserved." (Tim Minchin)

                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts   

Analysing residue names:
There are: 16343      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


...

No protocol specified
Reading file m.tpr, VERSION 2020.7 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 5 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Step -1, time -0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.130695, max 0.472812 (between atoms 7755 and 7754)
bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =          500
Step=    0, Dmax= 1.0e-02 nm, Epot=  8.65474e+10 Fmax= 1.13160e+13, atom= 8282
Step=    1, Dmax= 1.0e-02 nm, Epot=  2.70657e+10 Fmax= 1.83555e+12, atom= 8322
Step=    2, Dmax= 1.2e-02 nm, Epot=  1.05160e+10 Fmax= 1.98343e+11, atom= 3626
Step=    3, Dmax= 1.4e-02 nm,

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.321 nm, buffer size 0.221 nm
Set rlist, assuming 4x4 atom pair-list, to 1.226 nm, buffer size 0.126 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
This run will generate roughly 8 Mb of data


No protocol specified
Reading file r.tpr, VERSION 2020.7 (single precision)
Changing nstlist from 20 to 25, rlist from 1.226 to 1.28

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


Overriding thread affinity set outside gmx mdrun
starting mdrun 'INSANE! Membrane UpperLeaflet>DSPC:CAMP=80.0:20.0 LowerLeaflet>DSPC:CAMP=80.0:20.0'
2500000 steps,  50000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.986826 0.986826 0.988662
step 2499900, remaining wall clock time:     0 s          
Writing final coordinates.
step 2500000, remaining wall clock time:     0 s          
NOTE: 11 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.323 nm, buffer size 0.223 nm
Set rlist, assuming 4x4 atom pair-list, to 1.227 nm, buffer size 0.127 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup


Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to -25167497
Velocities were taken from a Maxwell distribution at 373 K
Cleaning up constraints and constant bonded interactions with virtual sites
Simulated annealing for group Lipids: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Simulated annealing for group Solvent: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Number of degrees of freedom in T-Coupling group Lipids is 22461.00
Number of degrees of freedom in T-Coupling group Solve

In [13]:
PC_comp = 80
ST_comp = 20

PC = 'DSPC'
ST = 'CAMP'


rep = 5

r = 2

os.chdir(out_put_dir_main)
outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
if os.path.isdir(outdir)==False:
    os.mkdir(outdir)
    os.chdir(outdir)
elif os.path.isdir(outdir)==True:
    os.chdir(outdir)

print ('Folder:', outdir)

print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
    -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
    -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))


prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

Folder: DSPC_80_CAMP_20_box_15_15_rep2
python2 /data1/lisbeth/PYTHON/insane.py -l DSPC:80 -l CAMP:20     -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top


; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 361 lipids in upper leaflet, 361 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 8352
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 8353 23993
; NDX System 1 23993
; "I mean, the good stuff is just INSANE" --Julia Ormond
                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul

Analysing residue names:
There are: 16361      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


Setting the LD random seed to -167805474
Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Cleaning up constraints and constant bonded interactions with virtual sites
Number of degrees of freedom in T-Coupling group System is 69384.00

Back Off! I just backed up memion_2.tpr to ./#memion_2.tpr.1#

GROMACS reminds you: "Let us not get carried away with our ideas and take our models too seriously" (Nancy Swanson)

                      :-) GROMACS - gmx genion, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feens

Analysing residue names:
There are: 16361      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


...

No protocol specified
Reading file m.tpr, VERSION 2020.7 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 5 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Step -1, time -0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.126287, max 0.453328 (between atoms 3629 and 3628)
bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length

Back Off! I just backed up m.trr to ./#m.trr.1#

Back Off! I just backed up m.edr to ./#m.edr.1#

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =          500
Step=    0, Dmax= 1.0e-02 nm, Epot=  1.93473e+11 Fmax= 3.08966e+13, atom= 8052
Step=    1, Dmax= 1.0e-02 nm, Epot=  4.47532e+10 Fmax= 5.71975e+12, atom= 3966
Step=    2

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.321 nm, buffer size 0.221 nm
Set rlist, assuming 4x4 atom pair-list, to 1.226 nm, buffer size 0.126 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
This run will generate roughly 8 Mb of data


No protocol specified
Reading file r.tpr, VERSION 2020.7 (single precision)
Changing nstlist from 20 to 25, rlist from 1.226 to 1.28

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Back Off! I just backed up r.xtc to ./#r.xtc.1#

Back Off! I just backed up r.edr to ./#r.edr.1#
starting mdrun 'INSANE! Membrane UpperLeaflet>DSPC:CAMP=80.0:20.0 LowerLeaflet>DSPC:CAMP=80.0:20.0'
2500000 steps,  50000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.986772 0.986772 0.988601
step 2499900, remaining wall clock time:     0 s          
Writing final coordinates.
step 2500000, remaining wall clock time:     0 s          
               Core t (s)   Wall t (s)        (%)
       Time:    89970.098     8997.010    

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.323 nm, buffer size 0.223 nm
Set rlist, assuming 4x4 atom pair-list, to 1.227 nm, buffer size 0.127 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup


Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to 2146797243
Velocities were taken from a Maxwell distribution at 373 K
Cleaning up constraints and constant bonded interactions with virtual sites
Simulated annealing for group Lipids: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Simulated annealing for group Solvent: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Number of degrees of freedom in T-Coupling group Lipids is 22461.00
Number of degrees of freedom in T-Coupling group Solv

In [8]:
PC_comp = 85
ST_comp = 15

PC = 'DSPC'
#ST = 'CAMP'

for ST in Sterols:

    for r in range(1,5):

        os.chdir(out_put_dir_main)
        outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
        if os.path.isdir(outdir)==False:
            os.mkdir(outdir)
            os.chdir(outdir)
        elif os.path.isdir(outdir)==True:
            os.chdir(outdir)

        print ('Folder:', outdir)

        print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
            -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
        os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
            -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))

        prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

Folder: DSPC_85_CHOL_15_box_15_15_rep1
python2 /data1/lisbeth/PYTHON/insane.py -l DSPC:85 -l CHOL:15             -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top


; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 361 lipids in upper leaflet, 361 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 8316
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 8317 23975
; NDX System 1 23975
; "I mean, the good stuff is just INSANE" --Julia Ormond
                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul

Analysing residue names:
There are: 16379      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


Reading file memion.tpr, VERSION 2020.7 (single precision)
No ions to add, will just copy input configuration.

Back Off! I just backed up memion.gro to ./#memion.gro.1#

GROMACS reminds you: "Like other defaulters, I like to lay half the blame on ill-fortune and adverse circumstances" (Mr. Rochester in Jane Eyre by Charlotte Bronte)

                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pasca

Analysing residue names:
There are: 16379      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


...

Reading file m.tpr, VERSION 2020.7 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 5 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Step -1, time -0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.123050, max 0.471504 (between atoms 8177 and 8176)
bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =          500
Step=    0, Dmax= 1.0e-02 nm, Epot=  7.93199e+09 Fmax= 4.77718e+11, atom= 8234
Step=    1, Dmax= 1.0e-02 nm, Epot=  6.39686e+09 Fmax= 4.52137e+11, atom= 4032
Step=    2, Dmax= 1.2e-02 nm, Epot=  3.09635e+09 Fmax= 3.34826e+10, atom= 8024
Step=    3, Dmax= 1.4e-02 nm, Epot=  1.41489e+09 Fm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.322 nm, buffer size 0.222 nm
Set rlist, assuming 4x4 atom pair-list, to 1.227 nm, buffer size 0.127 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
This run will generate roughly 8 Mb of data


Reading file r.tpr, VERSION 2020.7 (single precision)
Changing nstlist from 20 to 25, rlist from 1.227 to 1.281

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


Overriding thread affinity set outside gmx mdrun
starting mdrun 'INSANE! Membrane UpperLeaflet>DSPC:CAMP=85.0:15.0 LowerLeaflet>DSPC:CAMP=85.0:15.0'
2500000 steps,  50000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.986089 0.986089 0.987632
step 2499900, remaining wall clock time:     0 s          nish Tue Apr 11 04:47:20 2023
Writing final coordinates.
step 2500000, remaining wall clock time:     0 s          
NOTE: 13 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
    

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.323 nm, buffer size 0.223 nm
Set rlist, assuming 4x4 atom pair-list, to 1.228 nm, buffer size 0.128 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup


Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to -805341329
Velocities were taken from a Maxwell distribution at 373 K
Cleaning up constraints and constant bonded interactions with virtual sites
Simulated annealing for group Lipids: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Simulated annealing for group Solvent: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Number of degrees of freedom in T-Coupling group Lipids is 23325.00
Number of degrees of freedom in T-Coupling group Solv

In [5]:
PC_comp = 95
ST_comp = 5

PC = 'DSPC'
ST = 'CAMP'

#for ST in Sterols:

for r in range(3,5):

        os.chdir(out_put_dir_main)
        outdir = '{}_{}_{}_{}_box_15_15_rep{}'.format(PC, PC_comp,ST, ST_comp,r)
        if os.path.isdir(outdir)==False:
            os.mkdir(outdir)
            os.chdir(outdir)
        elif os.path.isdir(outdir)==True:
            os.chdir(outdir)

        print ('Folder:', outdir)

        print ('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
            -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))
        os.system('python2 {4}/insane.py -l {0}:{1} -l {2}:{3} \
            -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top'.format(PC, PC_comp, ST, ST_comp, insane))

        prepare_setup_hexa(ST, lipids='{0} {1}'.format(PC, ST))

Folder: DSPC_95_CAMP_5_box_15_15_rep3
python2 /data1/lisbeth/PYTHON/insane.py -l DSPC:95 -l CAMP:5             -pbc hexagonal -x 15 -y 15 -z 13 -sol W -o mem.gro -p topol.top


; X: 15.000 (19 bins) Y: 15.000 (19 bins) in upper leaflet
; X: 15.000 (19 bins) Y: 15.000 (19 bins) in lower leaflet
; 361 lipids in upper leaflet, 361 lipids in lower leaflet
; NDX Solute 1 0
; Charge of protein: 0.000000
; NDX Membrane 1 8568
; Charge of membrane: 0.000000
; Total charge: 0.000000
; NDX Solvent 8569 24295
; NDX System 1 24295
; "I mean, the good stuff is just INSANE" --Julia Ormond
                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul

Analysing residue names:
There are: 16447      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data


                      :-) GROMACS - gmx grompp, 2020.7 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    Peter Kasson        Jiri Kraus      Carsten Kutzner      Per Larsson    
  Justin A. Lemkul    Viveca Lindahl    Magnus Lundborg     Erik Marklund   
    Pascal Merz     Pieter Meulenhoff    Teemu Murtola       Szilard Pall   
    Sander Pronk      Roland Schulz      Michael Shirts    Alexey Shvetsov  
   Alfons Sijbers     Peter Tieleman      Jon Vincent      Teemu Virolainen 
 Christian Wennberg    Maarten Wolf      Artem Zhmurov   
                           and the project leaders:
     

Analysing residue names:
There are: 16447      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
This run will generate roughly 3 Mb of data
Will try to add 170 NA ions and 170 CL ions.
Select a continuous group of solvent molecules
Selected 4: 'W'

Processing topology
Replacing 340 solute molecules in topology file (topol.top)  by 170 NA and 170 CL ions.


...

Reading file m.tpr, VERSION 2020.7 (single precision)
1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 5 OpenMP threads 


Overriding thread affinity set outside gmx mdrun

Step -1, time -0.02 (ps)  LINCS WARNING
relative constraint deviation after LINCS:
rms 0.124888, max 0.408254 (between atoms 8521 and 8520)
bonds that rotated more than 30 degrees:
 atom 1 atom 2  angle  previous, current, constraint length

Steepest Descents:
   Tolerance (Fmax)   =  1.00000e+01
   Number of steps    =          500
Step=    0, Dmax= 1.0e-02 nm, Epot=  1.55508e+09 Fmax= 3.88958e+10, atom= 4264
Step=    1, Dmax= 1.0e-02 nm, Epot=  9.56031e+08 Fmax= 4.32866e+09, atom= 4274
Step=    2, Dmax= 1.2e-02 nm, Epot=  6.83415e+08 Fmax= 7.51099e+08, atom= 2105
Step=    3, Dmax= 1.4e-02 nm, Epot=  4.14688e+08 Fm

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.324 nm, buffer size 0.224 nm
Set rlist, assuming 4x4 atom pair-list, to 1.229 nm, buffer size 0.129 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup
This run will generate roughly 8 Mb of data


Reading file r.tpr, VERSION 2020.7 (single precision)
Changing nstlist from 20 to 25, rlist from 1.229 to 1.284

1 GPU selected for this run.
Mapping of GPU IDs to the 1 GPU task in the 1 rank on this node:
  PP:0
PP tasks will do (non-perturbed) short-ranged interactions on the GPU
PP task will update and constrain coordinates on the CPU
Using 1 MPI thread
Using 10 OpenMP threads 


Overriding thread affinity set outside gmx mdrun
starting mdrun 'INSANE! Membrane UpperLeaflet>DSPC:CAMP=95.0:5.0 LowerLeaflet>DSPC:CAMP=95.0:5.0'
2500000 steps,  50000.0 ps.

Step 0  Warning: pressure scaling more than 1%, mu: 0.98472 0.98472 0.986013
step 2499900, remaining wall clock time:     0 s          
Writing final coordinates.
step 2500000, remaining wall clock time:     0 s          
NOTE: 15 % of the run time was spent in pair search,
      you might want to increase nstlist (this has no effect on accuracy)

               Core t (s)   Wall t (s)        (%)
       Time:     7769.546      776.95

Determining Verlet buffer for a tolerance of 0.005 kJ/mol/ps at 373 K
Calculated rlist for 1x1 atom pair-list as 1.324 nm, buffer size 0.224 nm
Set rlist, assuming 4x4 atom pair-list, to 1.229 nm, buffer size 0.129 nm
Note that mdrun will redetermine rlist based on the actual pair-list setup


Generated 844 of the 356590 non-bonded parameter combinations
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'DSPC'
Excluding 1 bonded neighbours molecule type 'CAMP'
Excluding 1 bonded neighbours molecule type 'W'
Excluding 1 bonded neighbours molecule type 'NA'
Excluding 1 bonded neighbours molecule type 'CL'
Setting gen_seed to -135266461
Velocities were taken from a Maxwell distribution at 373 K
Cleaning up constraints and constant bonded interactions with virtual sites
Simulated annealing for group Lipids: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Simulated annealing for group Solvent: Single, 3 timepoints
Time (ps)   Temperature (K)
      0.0      373.0
1000000.0      273.0
2000000.0-     373.0
Number of degrees of freedom in T-Coupling group Lipids is 25053.00
Number of degrees of freedom in T-Coupling group Solv